<a href="https://colab.research.google.com/github/r-dube/fakejobs/blob/main/fj_nohidden.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Load the modules used
import numpy as np
import scipy as sci
import pandas as pd
import string
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# from sklearn.linear_model import LogisticRegression
import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [2]:
# For reproducible results
# except for variability introduced by GPU
import random as rn
import os
os.environ['PYTHONHASHSEED'] = '42'
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # avoid using GPU for reproducible results
np.random.seed(42)
rn.seed(42)
tf.random.set_seed(42)

In [3]:
# Set data_url, the location of the data
# Data is not loaded from a local file
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_small.csv"
# data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fj_medium.csv"
data_url="https://raw.githubusercontent.com/r-dube/fakejobs/main/data/fake_job_postings.csv"

In [4]:
def fj_load_df_from_url():
    """
    Load dataframe from csv file
    Input:
        None
    Returns:
        dataframe
    """

    df = pd.read_csv(data_url)

    print ('Loaded dataframe shape', df.shape)

    counts = fj_label_stats(df)
    print ('Not fraudulent', counts[0], 'Fraudulent', counts[1])

    print(df.describe())

    print ('NAs/NANs in data =>')
    print(df.isna().sum())

    return df

def fj_label_stats(df):
    """
    Very basic label statistics
    Input: 
        Dataframe
    Returns:
        Number of samples with 0, 1 as the label
    """
    counts = np.bincount(df['fraudulent'])
    return counts

def fj_txt_only(df):
    """
    Combine all the text fields, discard everything else except for the label
    Input: 
        Dataframe
    Returns:
        Processed dataframe
    """
    
    df.fillna(" ", inplace = True)

    df['text'] = df['title'] + ' ' + df['location'] + ' ' + df['department'] + \
    ' ' + df['company_profile'] + ' ' + df['description'] + ' ' + \
    df['requirements'] + ' ' + df['benefits'] + ' ' + df['employment_type'] + \
    ' ' + df['required_education'] + ' ' + df['industry'] + ' ' + df['function'] 

    del df['title']
    del df['location']
    del df['department']
    del df['company_profile']
    del df['description']
    del df['requirements']
    del df['benefits']
    del df['employment_type']
    del df['required_experience']
    del df['required_education']
    del df['industry']
    del df['function']  
    
    del df['salary_range']
    del df['job_id']
    del df['telecommuting']
    del df['has_company_logo']
    del df['has_questions']

    return df

In [5]:
df = fj_load_df_from_url()
df = fj_txt_only(df)
print('Maximum text length', df['text'].str.len().max())

Loaded dataframe shape (17880, 18)
Not fraudulent 17014 Fraudulent 866
             job_id  telecommuting  ...  has_questions    fraudulent
count  17880.000000   17880.000000  ...   17880.000000  17880.000000
mean    8940.500000       0.042897  ...       0.491723      0.048434
std     5161.655742       0.202631  ...       0.499945      0.214688
min        1.000000       0.000000  ...       0.000000      0.000000
25%     4470.750000       0.000000  ...       0.000000      0.000000
50%     8940.500000       0.000000  ...       0.000000      0.000000
75%    13410.250000       0.000000  ...       1.000000      0.000000
max    17880.000000       1.000000  ...       1.000000      1.000000

[8 rows x 5 columns]
NAs/NANs in data =>
job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telec

In [6]:
# train-test split
train_text, test_text, train_labels , test_labels = train_test_split(df['text'], df['fraudulent'] , test_size = 0.15)

In [7]:
# model 2: BOW + output node
cv = CountVectorizer(strip_accents='unicode', lowercase=True, stop_words='english', dtype=np.int8) 
cv_train_sparse = cv.fit_transform(train_text)
cv_train_dense = sci.sparse.csr_matrix.todense(cv_train_sparse)

cv_test_sparse = cv.transform(test_text)
cv_test_dense = sci.sparse.csr_matrix.todense(cv_test_sparse)
print('BOW for cv_train:', cv_train_dense.shape)
print('BOW for cv_test:', cv_test_dense.shape)

BOW for cv_train: (15198, 96150)
BOW for cv_test: (2682, 96150)


In [8]:
# no hidden layer model
model2 = Sequential()
model2.add(Dense(units = 1 , activation = 'sigmoid' , input_dim = cv_train_dense.shape[1]))
model2.compile(optimizer = 'adam' , loss = 'binary_crossentropy' , metrics = ['accuracy', tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])
model2.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 96151     
Total params: 96,151
Trainable params: 96,151
Non-trainable params: 0
_________________________________________________________________


In [9]:
model2.fit(cv_train_dense, train_labels, epochs = 30)

Epoch 1/30
475/475 [==============================] - 7s 12ms/step - loss: 0.1886 - accuracy: 0.9488 - false_positives: 10.2794 - false_negatives: 325.3908
Epoch 2/30
475/475 [==============================] - 6s 12ms/step - loss: 0.0538 - accuracy: 0.9843 - false_positives: 8.0105 - false_negatives: 111.2017
Epoch 3/30
475/475 [==============================] - 6s 12ms/step - loss: 0.0330 - accuracy: 0.9904 - false_positives: 6.9916 - false_negatives: 62.4265
Epoch 4/30
475/475 [==============================] - 6s 12ms/step - loss: 0.0231 - accuracy: 0.9939 - false_positives: 1.3109 - false_negatives: 42.5840
Epoch 5/30
475/475 [==============================] - 6s 12ms/step - loss: 0.0170 - accuracy: 0.9971 - false_positives: 1.5777 - false_negatives: 21.6366
Epoch 6/30
475/475 [==============================] - 6s 12ms/step - loss: 0.0132 - accuracy: 0.9981 - false_positives: 1.0315 - false_negatives: 14.4559
Epoch 7/30
475/475 [==============================] - 6s 12ms/step - loss

In [10]:
pred_soft2 = model2.predict(cv_test_dense)

In [11]:
# pred = np.around(pred_soft, decimals = 0)
pred = np.where(pred_soft2 > 0.15, 1, 0)

acc = accuracy_score(pred, test_labels)
f1 = f1_score(pred, test_labels)

cm = confusion_matrix(test_labels, pred)
tn = cm[0][0]
fn = cm[1][0]
tp = cm[1][1]
fp = cm[0][1]

print('Accuracy score: {:.4f}'.format(acc), 'F1 score: {:.4f}'.format(f1))
print('False Positives: {:.0f}'.format(fp), 'False Negatives: {:.0f}'.format(fn))
print('Confusion matrix:\n', cm)

Accuracy score: 0.9851 F1 score: 0.8551
False Positives: 14 False Negatives: 26
Confusion matrix:
 [[2524   14]
 [  26  118]]
